# Evaluating Coccinelle Semantic Patches in linux-mainline

### Setting up the notebook

In [1]:
# Import all needed libraries
import json
import calendar
import random
import os
import sys
from datetime import date, timedelta

import faker
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from delorean import parse
import matplotlib

# Enable inline plotting
%matplotlib inline

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.3 (v3.6.3:2c5fed8, Oct  3 2017, 18:11:49) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.20.3
Matplotlib version 2.1.0


### Preparing the data

In [3]:
# Dictionary listing all Semantic Patches in linux-4.14-rc6
# TODO: Should refactor into file 'cocci_linux_4_14_rc6.json'

cocci_linux_4_14_rc6 = {
    'info': {
        'repository': 'linux-4.14-rc6'
    },
    'scripts': [
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'alloc_cast.cocci',
            'confidence': 'High',
            'description': "Remove casting the values returned by memory allocation functions like "
                "kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. "
                "This makes an effort to find cases of casting of values returned "
                "by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, "
                "kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting "
                "as it is not required. The result in the patch case may need some reformatting."
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'kzalloc_simple.cocci',
            'confidence': 'High',
            'description': """use kzalloc rather than..."""
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'pool_zalloc_simple.cocci',
            'confidence': 'N.A.',
            'description': """Use *_pool_zalloc rather than..."""
        },
        # TODO
    ]
}

# cocci_linux_4_14_rc6

In [4]:
print(cocci_linux_4_14_rc6)
len(cocci_linux_4_14_rc6['scripts'])

{'info': {'repository': 'linux-4.14-rc6'}, 'scripts': [{'path': 'scripts/coccinelle/api/alloc', 'filename': 'alloc_cast.cocci', 'confidence': 'High', 'description': 'Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting.'}, {'path': 'scripts/coccinelle/api/alloc', 'filename': 'kzalloc_simple.cocci', 'confidence': 'High', 'description': 'use kzalloc rather than...'}, {'path': 'scripts/coccinelle/api/alloc', 'filename': 'pool_zalloc_simple.cocci', 'confidence': 'N.A.', 'description': 'Use *_pool_zalloc rather than...'}]}


3

### Creating the DataFrame for cocci_linux_4_14_rc6

In [5]:
data = cocci_linux_4_14_rc6['scripts']
df = DataFrame(data) # , index=cocci_linux_4_14_rc6['scripts']['filename']

#df.count()
#df.describe()
#json.dumps(data)
#print(df.index)
#print()
#print(df.filename)
#print()

df.head()

,confidence,description,filename,path
0,High,Remove casting the values returned by memory a...,alloc_cast.cocci,scripts/coccinelle/api/alloc
1,High,use kzalloc rather than...,kzalloc_simple.cocci,scripts/coccinelle/api/alloc
2,N.A.,Use *_pool_zalloc rather than...,pool_zalloc_simple.cocci,scripts/coccinelle/api/alloc


In [6]:
# Rearrange columns
cols = ['path', 'filename', 'confidence', 'description']
df = df[cols]

df

,path,filename,confidence,description
0,scripts/coccinelle/api/alloc,alloc_cast.cocci,High,Remove casting the values returned by memory a...
1,scripts/coccinelle/api/alloc,kzalloc_simple.cocci,High,use kzalloc rather than...
2,scripts/coccinelle/api/alloc,pool_zalloc_simple.cocci,N.A.,Use *_pool_zalloc rather than...


In [7]:
# Export the dataframe to an Excel file
#df.to_excel('test.xls')

### Creating DataFrames for instrument-coccicheck reports 

In [8]:
# Report against linux-4.14-rc6 full-tree

report_linux_4_14_rc6_fulltree = {
    'info': {
        'note': """Report against linux-4.14-rc6 full-tree""",
        'script_start': 'Thu Oct  5 13:11:29 CEST 2017',
        'src_sha': 'd81fa669e3de7eb8a631d7d95dac5fbcb2bf9d4e',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',         'L':  13, 'W':  0, 'E': 0},
        {'filename': 'kzalloc-simple.cocci',     'L':   4, 'W':  4, 'E': 0},
        {'filename': 'pool_zalloc-simple.cocci', 'L':  22, 'W': 22, 'E': 0}, 
        # TODO
    ]
}

In [9]:
data2_full = DataFrame(report_linux_4_14_rc6_fulltree['statistics'])
data2_full = data2_full[['filename','L', 'W', 'E']]

data2_full

,filename,L,W,E
0,alloc_cast.cocci,13,0,0
1,kzalloc-simple.cocci,4,4,0
2,pool_zalloc-simple.cocci,22,22,0


In [10]:
#data2_full.plot()
#data2_full[['L', 'W', 'E']].plot()
#data2_full[['filename','L', 'W', 'E']].plot(kind='bar')

#len(data2_full['filename'].unique())

In [11]:
# Report against linux-4.4.50 minimized-tree

report_linux_4_4_50_minimizedtree = {
    'info': {
        'note': """Report against linux-4.4.50 minimzed-tree""",
        'script_start': 'Mon Oct 23 16:29:52 IST 2017',
        'cocci_sha': '9e66317d3c92ddaab330c125dfe9d06eee268aff',
        'src_sha': '90dcab23bbc81fbfa25dfdb91d4ce974a69bd210',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',         'L':  2, 'W': 0, 'E': 0},
        {'filename': 'kzalloc-simple.cocci',     'L':  0, 'W': 0, 'E': 0},
        {'filename': 'pool_zalloc-simple.cocci', 'L':  5, 'W': 0, 'E': 0}, 
        # TODO
    ]
}